In [1]:
import pandas as pd

#reading the data 
data = pd.read_csv('healthcare-dataset-stroke-data.csv')
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [2]:
#checking the datas for NaN values
data.bmi.isnull().value_counts()
#observing data
data.tail()
data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [3]:
#Turning the objects into int
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data['gender'] = le.fit_transform(data['gender'])
data['Residence_type'] = le.fit_transform(data['Residence_type'])
data['ever_married'] = le.fit_transform(data['ever_married'])
data['smoking_status'] = le.fit_transform(data['smoking_status'])
data['work_type'] = le.fit_transform(data['work_type'])

In [6]:
#setting NaN column 
bmi = ['bmi']
data_bmi = data.loc[: , bmi]
data_bmi.head()

,bmi
0,36.6
1,NaN
2,32.5
3,34.4
4,24.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   int64  
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   int64  
 6   work_type          5110 non-null   int64  
 7   Residence_type     5110 non-null   int64  
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   int64  
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(9)
memory usage: 479.2 KB


In [10]:
#Using KNN classifier to train and replace bmi NaN values
from sklearn.impute import KNNImputer
import numpy as np
imp = KNNImputer(missing_values = np.nan, n_neighbors = 5)
#training the data with KNN
imp.fit(data_bmi)


KNNImputer()

In [112]:
#Replacing the NaN values with predicted values using transform based off of 5 nearest neighbors and creating a new DataFrame for it
data_bmi = imp.transform(data_bmi)
data_bmi = pd.DataFrame(data_bmi, columns = bmi)
data_bmi

,bmi
0,36.600000
1,28.893237
2,32.500000
3,34.400000
4,24.000000
...,...
5105,28.893237
5106,40.000000
5107,30.600000
5108,25.600000


In [11]:
data_bmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   bmi     4909 non-null   float64
dtypes: float64(1)
memory usage: 40.1 KB


In [ ]:
#adding back/ merging the bmi replaced datas back into the original Dataframe to get a clean data
data = data.merge(data_bmi)

In [120]:
#seperating x and y to train_test and split
factors = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'] 
x = data[factors]
data.stroke = data.stroke.astype(int)
y = data.stroke

TypeError: 'NoneType' object is not subscriptable

In [121]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(x,y, random_state = 1)

In [122]:
'''
#Got 90.5% accuracy on DecisionTree
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state = 1)
model.fit(train_x, train_y)
prediction =  model.predict(val_x)
prediction = prediction.astype(int)
'''

'\n#Got 90.5% accuracy on DecisionTree\nfrom sklearn.tree import DecisionTreeRegressor\nmodel = DecisionTreeRegressor(random_state = 1)\nmodel.fit(train_x, train_y)\nprediction =  model.predict(val_x)\nprediction = prediction.astype(int)\n'

In [123]:
#using RandomForest to fit and predict the accuracy
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state = 1)
rf_model.fit(train_x, train_y)
val_prediction = rf_model.predict(val_x)
val_prediction = val_prediction.astype(int)

In [109]:
from sklearn.metrics import accuracy_score
#tree = accuracy_score(val_y, prediction)
forest = accuracy_score(val_y, val_prediction)
print(f"RandomForest accuracy: {forest}")

RandomForest accuracy: 0.9413145539906104


In [118]:
print(data)

None
